### 페이지를 불러온다.

In [12]:
import random
import requests
import os
from fake_useragent import UserAgent
import json
from bs4 import BeautifulSoup
import time
import copy
import pandas as pd
import os 

ua = UserAgent()
header = {
    'User-Agent' : str(ua.chrome),
    'Referer' : "http://srchdb1.chosun.com.access.ewha.ac.kr/pdf/i_archive/searchList.jsp"
}

url = "http://srchdb1.chosun.com.access.ewha.ac.kr/pdf/i_archive/searchList.jsp"
resultList = []

def startSearchNewsByPage(startDate : int, endDate : int,keyword : str, pageNum : int):
    
    data = {
        "currentPage" : "{}".format(pageNum),
        "sRowsperPage" : "300",
        "searchPage" : "", 
        "FV" : "{}".format(keyword),
        "INDEX_FV_VALUE" : "TI.TX.KW",
        "AU_FV" : "", 
        "PD_TYPE" : False,
        "PD_F0" : "all",
        "PD_F1" : "{}".format(startDate), 
        "PD_OP" : "1",
        "PD_F2" : "{}".format(endDate),
        "DATA_SORT" : "1",
        "LIST_TYPE" : True,
        "PP_F1" : "" 
    }
    
    res = requests.post(url, headers=header, data=data).text
    parseHTML(res)
    
def parseHTML(resTxt : str):
    dom = BeautifulSoup(resTxt, 'html.parser')
    tables = dom.select('table')
    accepted_idx = 11
    tmpDict = {} # 여기에 임시로 데이를 담아놓는다. 

    for idx, item in enumerate(tables):

        if idx != accepted_idx:
            continue
        else:

            convertedIdx = (idx - 11) % 3

            # 제목
            if convertedIdx == 0:

                if len(item.find_all("font", {"class": "wiz301"})) == 0:
                    
                    break

                title = item.find_all("font", {"class": "wiz301"})[0].text.replace("\n\n","").replace("\r\n\r\n\t\t\n","")
                #print("{}".format(title.replace('\t',"")))
                
                tmpDict["title"] = title

                accepted_idx = accepted_idx + 1
            # 
            elif convertedIdx == 1:

                #print(item.find_all("a",href=True)[0].attrs["href"])
                pdfLink = item.find_all("a",href=True)[0].attrs["href"]
                date = item.text.split(" ")[2]
                #print("{}".format(date))
                accepted_idx = accepted_idx + 2
                
                tmpDict["date"] = date
                tmpDict["pdfLink"] = pdfLink
                
                resultList.append(copy.deepcopy(tmpDict))
                
                tmpDict.clear()
                

def makeExcelFile(fileName : str):
    
    #기존에 존재하던 파일을 삭제한다.     
    resultFilePath = os.getcwd() + os.sep + "result.xlsx"
    if os.path.isfile(resultFilePath):
        os.remove(resultFilePath)

    
    df = pd.DataFrame(resultList)
    #엑셀 파일로 만든다. 
    df.to_excel("{}".format(fileName), # directory and file name to write
            sheet_name = 'Sheet1', 
            na_rep = 'NaN', 
            float_format = "%.2f", 
            header = True, 
            #columns = ["group", "value_1", "value_2"], # if header is False
            index = True, 
            index_label = "id", 
            startrow = 1, 
            startcol = 1, 
            #engine = 'xlsxwriter', 
            freeze_panes = (2, 0))
    print("엑셀 파일이 만들어졌습니다!")

for idx in range(1,4):
    startSearchNewsByPage(19910101,20191231,"ODA OR 공적개발원조 OR 정부개발원조 OR 개발원조 OR 해외원조 OR 국제개발협력 not 선정",idx)
    randomTime = random.uniform(0.0, 1.0)
    print("randomTime : {}".format(randomTime))
    time.sleep(randomTime)

makeExcelFile("result.xlsx")

randomTime : 0.030217531404714837
randomTime : 0.31853634744732373
randomTime : 0.45171572318003805
엑셀 파일이 만들어졌습니다!
